In [1]:
pip install pandas numpy matplotlib seaborn scikit-learn


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import StandardScaler


In [10]:
# Merge data
data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Aggregate customer metrics
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'ProductID': 'nunique',
    'TransactionID': 'count'
}).rename(columns={'TotalValue': 'TotalSpent', 'ProductID': 'UniqueProducts', 'TransactionID': 'TotalTransactions'})

print(customer_features.head())


            TotalSpent  UniqueProducts  TotalTransactions
CustomerID                                               
C0001          3354.52               5                  5
C0002          1862.74               4                  4
C0003          2725.38               4                  4
C0004          5354.88               8                  8
C0005          2034.24               3                  3


In [12]:
# Ensure n_neighbors is within valid range
n_neighbors = min(len(scaled_features), 4)

model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
model.fit(scaled_features)

lookalikes = {}
for i, customer in enumerate(customer_features.index[:20]):
    # Fetch neighbors for the current customer
    distances, indices = model.kneighbors([scaled_features[i]])

    # Avoid out-of-bound errors and self-match
    lookalikes[customer] = [
        (customer_features.index[j], distances[0][k])
        for k, j in enumerate(indices[0]) if j != i
    ]

print(lookalikes)


{'C0001': [('C0137', 0.003667657454288764), ('C0152', 0.013095351798187771), ('C0056', 0.06957342601465233)], 'C0002': [('C0029', 0.0002422080031764251), ('C0199', 0.00065307902551337), ('C0010', 0.0008181996361311672)], 'C0003': [('C0178', 5.081272465734532e-05), ('C0112', 0.0004295745385458627), ('C0131', 0.0004301212729144188)], 'C0004': [('C0021', 9.707724585339417e-05), ('C0075', 0.00022513237349808257), ('C0108', 0.0006845703848219475)], 'C0005': [('C0073', 3.353485602908268e-05), ('C0144', 4.643260952008621e-05), ('C0095', 5.336776390696407e-05)], 'C0006': [('C0079', 0.00010308278191162312), ('C0117', 0.008447647737665709), ('C0196', 0.04544005373437299)], 'C0007': [('C0085', 2.3800428107234595e-05), ('C0193', 0.000677995537777587), ('C0120', 0.0008071473753447078)], 'C0008': [('C0194', 0.00063301940126681), ('C0179', 0.001313500951214297), ('C0139', 0.002303421283235796)], 'C0009': [('C0077', 6.415667527348567e-05), ('C0142', 0.0003577859788136406), ('C0032', 0.0008537553493264